In [2]:
!pip install scikit-learn

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 17.2 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 37.7/37.7 MB 68.3 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4/4 [scikit-learn] [scikit-learn]


In [3]:
!pip install fastapi "uvicorn[standard]" scikit-learn pandas

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 10.8 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8/8 [fastapi]m7/8 [fastapi]e]]nv]


In [4]:
import pickle
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression

In [5]:
# Exemplo de dados
data = {
    "idade": [25, 45, 30, 50, 40, 35, 28, 52, 33, 41],
    "genero": ["M","F","M","F","M","M","F","F","M","F"],
    "tempo_de_contrato_meses": [12, 24, 5, 36, 48, 20, 6, 50, 15, 22],
    "gasto_mensal": [120, 200, 80, 300, 150, 110, 95, 310, 100, 180],
    "churn": [0,1,0,1,0,0,1,1,0,1]
}
df = pd.DataFrame(data)

X = df.drop(columns=["churn"])
y = df["churn"]

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42, stratify=y
)

# Definição de features
num_features = ["idade", "tempo_de_contrato_meses", "gasto_mensal"]
cat_features = ["genero"]

# Pré-processamento
preprocess = ColumnTransformer(
    transformers=[
        ("num", StandardScaler(), num_features),
        ("cat", OneHotEncoder(drop="first", handle_unknown="ignore"), cat_features),
    ]
)

# Pipeline completo
clf = Pipeline(steps=[
    ("preprocess", preprocess),
    ("model", LogisticRegression(max_iter=1000))
])

# Treino
clf.fit(X_train, y_train)

# Salvar pipeline
with open("pipeline.pkl", "wb") as f:
    pickle.dump(clf, f)


In [6]:
from joblib import dump, load

In [7]:
dump(clf, 'model.joblib')

['model.joblib']

In [ ]:
#Run Test
uvicorn app:app --host 0.0.0.0 --port 5000 --workers 2

#Test
curl -X POST http://127.0.0.1:5000/predict \
  -H "Content-Type: application/json" \
  -d '{
        "records": {
          "idade": 30,
          "genero": "M",
          "tempo_de_contrato_meses": 10,
          "gasto_mensal": 100
        }
      }'

#Output
[{"churn_probability":0.30353036311804166,"prediction":0}]